# 1. Librerias

In [3]:
import pandas as pd
import json
import numpy as np
import athena_utils as au

# 2. Query

In [4]:
# variables
tally=True
project_id='72, 73'
tag_test='Cuestionario de Salida - Grupo Control'

In [17]:
query_tally=fr"""

    SELECT DISTINCT
  -- Campos de proyecto
  CASE
    WHEN sss.content_type_associated_id = 267 THEN sss.object_id_associated
    ELSE NULL
  END AS project_id,
  
  -- Campos de usuario/estudiante
  au.user_mdl_id AS moodle_user_id,
  coalesce(CAST(ss.id AS VARCHAR), respondent_id) AS student_id,
  coalesce(CONCAT(ss.first_name, ' ', ss.last_name), respondent_id) AS student_name,
  
  -- Campos de enrollment
  coalesce(ee.institution, '-') AS educative_institution,
  coalesce(CONCAT(ee.grade, '+', ee.group_section), '-') AS grade_section,
  
  -- Campos de encuesta/evaluación
  sss.id AS unique_id,
  e.form_name AS evaluation_name,
  
  -- Campos de pregunta
  q.question_text AS question_name,
  CAST(NULL AS VARCHAR) AS tag_question,  -- ✅ CAST explícito
  q.question_id,
  q.question_text AS question,
  
  -- Campos de respuesta
  r.answer,
  CAST(NULL AS VARCHAR) AS right_answer,  -- ✅ CAST explícito
  
  -- Campos adicionales
  ssft.name AS tipo_test,
  au.id AS response_user_id

FROM
  datalake.survey_forms e 
  LEFT JOIN datalake.survey_questions q ON q.unique_id = e.unique_id
  LEFT JOIN datalake.survey_responses r ON r.question_id = q.question_id AND r.unique_id = q.unique_id
  left join datalake.survey_surveyformtally ssft ON e.element_id = ssft.tally_id
  LEFT JOIN datalake.survey_surveyscheduler_template ssst ON ssft.template_id = ssst.surveytemplate_id
  LEFT JOIN datalake.survey_surveyscheduler sss ON sss.id = ssst.surveyscheduler_id

  LEFT JOIN datalake.account_user au ON CAST(au.user_mdl_id AS VARCHAR) = r.moodle_id
  
  -- JOINs adicionales para obtener los campos necesarios
  LEFT JOIN datalake.student_student ss ON ss.user_mdl_id = au.user_mdl_id
  LEFT JOIN datalake.room_room_students rrs ON rrs.student_id = ss.id
  LEFT JOIN datalake.room_room rr ON rr.id = rrs.room_id
  LEFT JOIN datalake.enrollment_enrolment ee ON (
    (ee.group_id = rr.group_id OR ee.room_id = rr.id)
    AND ee.student_id = ss.id
    AND ee.state <> 'cancel'
    AND ee.state <> 'inactive'
  )
  LEFT JOIN datalake.projects p ON p.id = ee.b2b_project_id

WHERE
  r.answer IS NOT NULL  -- Solo respuestas completadas
  and e.form_name = 'Cuestionario de Salida - Grupo Control'

  
"""

In [6]:
query=f'''
SELECT DISTINCT
     p.id project_id
   , p.name project_name
   , me.moodle_id moodle_user_id
   , ss.id student_id
   , CONCAT(ss.first_name, ' ', ss.last_name) student_name
   , ee.institution educative_institution
   , coalesce(concat(ee.grade,'+', ee.group_section), '-') grade_section
   , rr.id room_id
   , ce.unique_id evaluation_unique_id
   , ce.name evaluation_name
   , ceq.name question_name
   , ceq.tag tag_question
   , ceq.question_id question_id
   , ceq.question_name question
   , cer.answer answer
   , cer.right_answer right_answer
   , ce.tag AS tipo_test

FROM
moodle_enrollment me
LEFT JOIN moodle_course_evaluations ce ON (me.course_id = ce.course_id)
LEFT JOIN moodle_course_evaluation_questions ceq ON (ce.unique_id = ceq.unique_id) AND ((ceq.question_name <> 'label') OR (ceq.question_name IS NULL))
LEFT JOIN moodle_course_evaluation_responses cer ON ((cer.unique_id = ceq.unique_id) AND (ceq.question_id = cer.question_id) AND (me.moodle_id = cer.moodle_id) AND (ce.type <> 'assign')  AND (cer.attempt_time_finish IS NOT NULL))
INNER JOIN room_room rr ON (rr.course_mdl_id = me.course_id)
LEFT JOIN student_student ss ON (ss.user_mdl_id = me.moodle_id)
LEFT JOIN room_room_students rrs ON ((rrs.student_id = ss.id) AND (rrs.room_id = rr.id))
LEFT JOIN enrollment_enrolment ee ON (((ee.group_id = rr.group_id) OR (ee.room_id = rr.id)) AND (ee.student_id = ss.id) AND (ee.state <> 'cancel') AND (ee.state <> 'inactive'))
LEFT JOIN projects p ON (p.id = ee.b2b_project_id)

where p.id={project_id} and ce.tag = '{tag_test}' 
-- and ce.name='Encuesta de satisfacción 2'

'''

In [18]:
if tally:
    df=au.run_athena_query(query_tally, 'pivot_response')
else:
    df=au.run_athena_query(query, 'pivot_response')

# 3. Funciones

In [19]:
def limpiar_texto(texto):

    if pd.isna(texto):
        return ''
    # Reemplaza tabuladores, retornos de carro y saltos de línea por un espacio
    texto = re.sub(r'[\t\r\n]', ' ', str(texto))
    # Reemplaza múltiples espacios por uno solo
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def ajustar_titulo(titulo, largo_primera_linea=44, largo_otras_lineas=40, max_lineas=2):

    # Si solo se permite una línea, truncar y rellenar
    if max_lineas == 1:
        if len(titulo) > largo_primera_linea:
            return titulo[:largo_primera_linea - 3] + "..."
        return titulo.center(largo_primera_linea)

    # Separar la primera línea con el largo específico
    if len(titulo) <= largo_primera_linea:
        linea1 = titulo.center(largo_primera_linea)
        return linea1
    else:
        linea1 = titulo[:largo_primera_linea]
        resto = titulo[largo_primera_linea:]

    # Envolver el resto en líneas más cortas
    lineas_extra = textwrap.wrap(resto.strip(), width=largo_otras_lineas)

    # Limitar la cantidad de líneas totales
    lineas_extra = lineas_extra[:max_lineas - 1]
    
    # Si hay más texto que lo permitido, truncar la última línea
    if len(lineas_extra) == (max_lineas - 1) and len(titulo) > len(linea1) + sum(len(l) for l in lineas_extra):
        if len(lineas_extra[-1]) > largo_otras_lineas - 3:
            lineas_extra[-1] = lineas_extra[-1][:largo_otras_lineas - 3] + "..."

    # Rellenar las líneas
    linea1 = linea1.center(largo_primera_linea)
    lineas_extra = [l.center(largo_otras_lineas) for l in lineas_extra]

    return "\n".join([linea1] + lineas_extra)

def ajustar_etiquetas(texto, max_length=30):
    """Ajusta etiquetas largas dividiéndolas en líneas y truncando si es necesario."""
    if len(texto) <= max_length:
        return texto
    # Dividir en líneas de máximo 30 caracteres
    lineas = textwrap.fill(texto, width=max_length, max_lines=2, placeholder="...").split('\n')
    return '\n'.join(lineas)

def extract_number_text(answer: str):
    '''Esta funcion separa la parte numerica y la string'''
    _pattern = re.compile(r'^\s*(\d+)(?:[\.\)\-\:\s]+)(.*)$')
    answer = str(answer).strip()
    m = _pattern.match(answer)
    if m:
        numero = m.group(1)
        texto  = m.group(2).strip()
        return numero, texto
    # si no coincide, devolvemos None y el texto completo
    return None, answer

def normalize_answer(row):
    if pd.notnull(row['Number']):
        question = row['question']
        number = row['Number']
        standard_text = mapping.get((question, number), row['Text'])
        return f"{number}. {standard_text}"
    return row['answer']

def normalize_question(row):
    question = row['question']
    tag = row['tag_question']
    standard_text = mapping.get(tag, question)
    return standard_text

def clasificar_tipo_pregunta(respuesta):
    if pd.isna(respuesta) or not isinstance(respuesta, str):
        return "Abierta"
    
    # 1) Limpiar espacios
    respuesta_limpia = respuesta.strip()
    baja = respuesta_limpia.lower()
    
    # 2) Verdadero/Falso como categóricas
    if baja in {"verdadero", "falso"}:
        return "Categorica"
    
    # 3) Si no hay ';', es abierta o categórica simple según prefijo
    if ';' not in respuesta_limpia:
        if re.match(r'^(\d+\.\s*|[A-Z][\-\)])', respuesta_limpia):
            return "Categorica"
        else:
            return "Abierta"
    
    # 4) Con ';', buscamos MULTISELECCION ESTRUCTURADA:
    patrones_multiseleccion = [
        r'\b\d+\.\s+[^\;]+(?:;\s*\d+\.\s+[^\;]+)+',     # 1. ...; 2. ...
        r'\b[A-Z]\)\s+[^\;]+(?:;\s*[A-Z]\)\s+[^\;]+)+', # A) ...; B) ...
        r'\b[A-Z]-\s+[^\;]+(?:;\s*[A-Z]-\s+[^\;]+)+'    # A- ...; B- ...
    ]
    for patron in patrones_multiseleccion:
        if re.search(patron, respuesta_limpia):
            return "Categorica Multiseleccion"
    
    # 5) Si arranca como categórica pero no multiselección:
    if re.match(r'^(\d+\.\s*|[A-Z][\-\)])', respuesta_limpia):
        return "Categorica"
    
    # 6) Lo que quede: abierta
    return "Abierta"

def consolidar_tipo_pregunta(series_tipos):
    """
    Serie de valores: ["Abierta", "Categorica", "Categorica Multiseleccion"].
    Devuelve:
      - "Categorica Multiseleccion" si al menos una respuesta es multiselección.
      - "Categorica" si todas las respuestas son categóricas.
      - "Abierta" en cualquier otro caso (incluye mixtas o que contengan abiertas).
    """
    tipos = set(series_tipos)
    if "Categorica Multiseleccion" in tipos:
        return "Categorica Multiseleccion"
    elif tipos == {"Categorica"}:
        return "Categorica"
    else:
        return "Abierta"

def extraer_numero(texto):
    """
    Intenta extraer un número entero al inicio del texto, justo antes de un punto.
    Retorna el número si se encuentra o None si no hay coincidencia.
    """
    # Se utiliza una expresión regular que busca dígitos seguidos de un punto al inicio del string
    m = re.match(r'\s*(\d+)\.', str(texto))
    if m:
        return int(m.group(1))
    return None

def un_tag_una_pregunta(dataframe: pd.DataFrame):
    '''Crear un diccionario con la primera pregunta por cada tag_question '''

    primera_pregunta_por_tag = (
        dataframe.groupby('tag_question')['question']
        .first()
        .to_dict()
    )

    # Reemplazar todas las preguntas con la seleccionada para ese tag_question
    dataframe['question'] = dataframe['tag_question'].map(primera_pregunta_por_tag)

    return dataframe

def renombrar_tipo_test(tipo_test):
    renombrar_test = {
        'cuestionario de entrada': 'Cuestionario de entrada',
        'cuestionario medio': 'Cuestionario medio',
        'cuestionario de salida': 'Cuestionario de salida',
        'examen de casos inicial': 'Ex. casos inicial',
        'examen de casos final': 'Ex. casos final',
        'examen final': 'Ex. final',
        'cuestionario de satisfacción modular': 'Satisfacción Modular',
        'cuestionario de satisfacción final': 'Satisfacción Final',
    }
    return renombrar_test.get(tipo_test, 'Otro')  # Devuelve Otro si no está en el diccionario

def ordenar_tipo_test(tipo_test):
    orden_test = {
        'cuestionario de entrada': 1,
        'cuestionario medio': 2,
        'cuestionario de salida': 3,
        'examen de casos inicial': 4,
        'examen de casos final': 5,
        'examen final': 6,
        'cuestionario de satisfacción modular': 7,
        'cuestionario de satisfacción final': 8,
    }
    return orden_test.get(tipo_test, 6)  # Devuelve 6 si no está en el diccionario

def ordenar_tag(tag_question):
    orden_tag = {
        "nombre": 1,
        "genero": 2,
        "correo_personal": 3,
        "celular": 4,
        "celular_de": 5,
        "tipo_documento": 6,
        "documento": 7,
        "nacimiento": 8,
        "etnia": 9,
        "nacionalidad": 10,
        "estrato_socioeconomico": 11,
        "nivel_educativo_familia": 12,
        "trabajar_ayuda_casa": 13,
        "cuidar_ayuda_casa": 14,
        "interes_tecnologia": 15,
        "dispositivos": 16,
        "forma_conectividad": 17,
        "uso_tecnologia_dia_a_dia": 18,
        "uso_tecnologia_a_futuro": 19,
        "planes_futuro": 20,
        "abandonar_estudios": 21,
        "motivo_abandonar_estudios": 22,
        "programas_educativos": 23,
        "prioridad_programas_educativos": 24,
        "trabajo": 25,
        "intereses_futuro": 26,
        "apoyo_financiero": 27,
        "financiamiento": 28,
        "cv": 29,
        "cv_experiencia_laboral": 30,
        "empresas": 31,
        "sitios_busqueda_laboral": 32,
        "retos_mercado_laboral": 33,
        "actividades_ultimo_anio": 34,
        "fuentes_informacion": 35,
        "fuente_informacion_otros": 36,
        "motivacion_familia": 37,
        "motivacion_familia_quienes": 38,
        "apoyo_econ_familia_estudio": 39,
        "actividades_profesores": 40,
        "apoyo_metas_prof_familia": 41,
        "ayuda_familiar_trabajos": 42,
        "motivacion_familiar_trabajo": 43,
        "motivacion_familiar_emprender": 44,
        "motivacion_profesores_metas": 45,
        # Competencias
        'innovacion':46, 
        'analisis':47,
        'critico':48,
        'comunicacion':49,
        'autogestion':50,
        'equipo':51,
    }
    return orden_tag.get(tag_question, 52)  # Devuelve 52 si no está en el diccionario


# 4. Normalizar

In [20]:
# ordernar el DataFrame por tipo de test
df['tipo_test_orden'] = df['tipo_test'].apply(ordenar_tipo_test)

df['tag_question_orden'] = df['tag_question'].apply(ordenar_tag)

df.sort_values(by=['project_id', 'tipo_test_orden','tag_question_orden'], inplace=True)

df.drop(columns=['tipo_test_orden', 'tag_question_orden'], inplace=True)

df['tipo_test'] = df['tipo_test'].apply(renombrar_tipo_test)

In [21]:
for col in ['answer', 'right_answer']: # Limpiar saltos de línea, retornos de carro y dobles comas en 'answer' y 'right_answer'
    df[col] = df[col].str.replace('\n', '', regex=False)
    df[col] = df[col].str.replace('\r', '', regex=False)
    df[col] = df[col].str.replace(',,', ',', regex=False)
    df[col] = df[col].str.strip()

#df['answer'] = df['answer'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # Elimino los acentos y caracteres especiales de las respuestas

df['question']=df['question'].str.replace('&nbsp;',' ', regex=False) # Elimino los &nbsp;

In [ ]:
# Pivotear el dataframe
df_pivoted = df.pivot_table(
    index=['student_name', 'educative_institution', 'grade_section'],
    columns='question',
    values='answer',
    aggfunc='first'  # Usar 'first' para evitar duplicados
).reset_index()

# Obtener el orden original de las preguntas
orden_preguntas = df['question'].unique()

# Reordenar las columnas respetando el orden original
question_cols = [col for col in orden_preguntas if col in df_pivoted.columns]
ordered_cols = ['student_name', 'educative_institution', 'grade_section'] + question_cols
df_pivoted = df_pivoted[ordered_cols]
# Mostrar el resultado


In [14]:
# Antes del pivot - verifica cuántos estudiantes tienes
print(f"Total de registros en df: {len(df)}")
print(f"Estudiantes únicos en df: {df['student_name'].nunique()}")
print(f"Registros con answer no-nulo: {df['answer'].notna().sum()}")
print(f"Estudiantes con al menos una respuesta: {df[df['answer'].notna()]['student_name'].nunique()}")

# Estudiantes sin respuestas
estudiantes_sin_respuesta = df[df['answer'].isna()]['student_name'].unique()
print(f"\nEstudiantes sin ninguna respuesta: {len(estudiantes_sin_respuesta)}")

Total de registros en df: 52863
Estudiantes únicos en df: 1
Registros con answer no-nulo: 52863
Estudiantes con al menos una respuesta: 1

Estudiantes sin ninguna respuesta: 0


# 4. Resultado

In [ ]:
df_pivoted.to_excel(f'{tag_test}_{project_id}.xlsx', index=False)